In [1]:
#TEST RESULTS (MODEL 30 SEC). This code produces the predictions 
#for the annotations of valence and arousal

This code performs predictions on the test data for each scenario.

Here is a detailed explanation for scenario 2. 
Scenario 2 uses the Random Forest and SVR_RBF, respectively, both trained on the fizz-emg-sub-vid features. The code performs the following steps:

1. Loop through all the folds (0 to 4) for scenario 2.
2. Load the arousal and valence models for each fold using joblib.
3. Set the test data folder for physiology data and annotations folder for the ground truth annotations.
4. Create a folder to save the results.
5. Loop through the test physiology files ending with "_resampled_1hz.csv".
6. Load the corresponding ground truth annotations file for the current test file.
7. Preprocess the test data for both arousal and valence by handling NaN values, resampling the data to match the annotations length, and selecting the fizz-emg-sub-vid features.
8. Use the loaded arousal and valence models to predict arousal and valence values for the test data.
9. Add the predicted arousal and valence values to the ground truth annotations DataFrame.
10. Save the resulting DataFrame with the predicted arousal and valence values to the specified folder.

In summary, this code snippet predicts arousal and valence values for the test data in each scenario using the best models for valence and arousal and saves the predictions in the specified folder.

Nota Bene:
The code starts by specifying the feature of interest for each models and copies the pipeline helper.

In [ ]:
#columns of interest

fizz_columns_of_interest = [
    'PPG_Clean', 'PPG_Rate', 'PPG_Peaks',
    'ECG_Rate','ECG_Quality',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'corrugator_EMG_Clean',
    'corrugator_EMG_Amplitude', 'corrugator_EMG_Activity',
    'corrugator_EMG_Onsets', 'corrugator_EMG_Offsets',
    'trapezius_EMG_Clean', 'trapezius_EMG_Amplitude',
    'trapezius_EMG_Activity', 'trapezius_EMG_Onsets',
    'trapezius_EMG_Offsets', 'zygomaticus_EMG_Clean',
    'zygomaticus_EMG_Amplitude', 'zygomaticus_EMG_Activity',
    'zygomaticus_EMG_Onsets', 'zygomaticus_EMG_Offsets',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    #'sub_num','vid_num',
]

fizz_columns_of_interest = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'corrugator_EMG_Clean',
    'corrugator_EMG_Amplitude', 'corrugator_EMG_Activity',
    'corrugator_EMG_Onsets', 'corrugator_EMG_Offsets',
    'trapezius_EMG_Clean', 'trapezius_EMG_Amplitude',
    'trapezius_EMG_Activity', 'trapezius_EMG_Onsets',
    'trapezius_EMG_Offsets', 'zygomaticus_EMG_Clean',
    'zygomaticus_EMG_Amplitude', 'zygomaticus_EMG_Activity',
    'zygomaticus_EMG_Onsets', 'zygomaticus_EMG_Offsets',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    # 'sub_num',
    # 'vid_num',
]

fizz_columns_of_interest_no_emg = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    # 'sub_num',
    # 'vid_num',
]

fizz_columns_of_interest_no_emg_sub_vid = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    'sub_num',
    'vid_num',
]

fizz_columns_of_interest_sub_vid = [ 
    'PPG_Rate', 
    'ECG_Rate','ECG_Quality',
    'EDA_Clean', 'EDA_Tonic', 'EDA_Phasic', 'SCR_Onsets', 'SCR_Peaks','SCR_Height', 'SCR_Amplitude',
    'RSP_Amplitude', 'RSP_Rate',
    'corrugator_EMG_Clean',
    'corrugator_EMG_Amplitude', 'corrugator_EMG_Activity',
    'corrugator_EMG_Onsets', 'corrugator_EMG_Offsets',
    'trapezius_EMG_Clean', 'trapezius_EMG_Amplitude',
    'trapezius_EMG_Activity', 'trapezius_EMG_Onsets',
    'trapezius_EMG_Offsets', 'zygomaticus_EMG_Clean',
    'zygomaticus_EMG_Amplitude', 'zygomaticus_EMG_Activity',
    'zygomaticus_EMG_Onsets', 'zygomaticus_EMG_Offsets',
    'RSP_RVT', 'RSP_Phase', 'RSP_Phase_Completion',
    'RSP_Symmetry_PeakTrough', 'RSP_Symmetry_RiseDecay','skt',
    'sub_num',
    'vid_num',
]

zipped_xs = list(zip(['fizz','fizz-emg','fizz-emg_sub_vid','fizz_sub_vid'],[fizz_columns_of_interest,
     fizz_columns_of_interest_no_emg,
     fizz_columns_of_interest_no_emg_sub_vid,
     fizz_columns_of_interest_sub_vid]))

In [ ]:
#copying the pipeline helper


import glob,os
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.metrics import mean_squared_error, r2_score
from scipy.signal import resample


def get_file_paths(subjects, video_numbers, x_folder, y_folder, file_type = '1hz_zscored'):
    x_files = []
    y_files = []

    for subject in subjects:
        for video in video_numbers:
            x_pattern = os.path.join(x_folder,
                                     f"sub_{int(subject)}_vid_{int(video)}_resampled_{file_type}.csv")
            y_pattern = os.path.join(y_folder, f"sub_{int(subject)}_vid_{int(video)}.csv")
            #print(x_pattern)

            x_files.extend(glob.glob(x_pattern))
            y_files.extend(glob.glob(y_pattern))

    return x_files, y_files


import torch
from torch.utils.data import Dataset

class EmognitionDataset(Dataset):
    def __init__(self, x_files, y_files, x_columns=None, y_columns=None, downsampling_factor_x=1,downsampling_factor_y=1, upsample_factor_x=1, upsample_factor_y=1):
        self.x_data = []
        self.y_data = []  
        self.sub_num_data = []
        self.vid_num_data = []

        for file in x_files:
            x_df = pd.read_csv(file)
            x_array = x_df[x_columns].values[::downsampling_factor_x]
            if x_array.shape[0] == 1:
                x_array = np.nan_to_num(x_array)
            
            if upsample_factor_x > 1:
                x_array = self.upsample(x_array, upsample_factor_x)
            
            self.x_data.append(x_array)

            sub_num_array = x_df['sub_num'].values[::downsampling_factor_x]
            vid_num_array = x_df['vid_num'].values[::downsampling_factor_x]
            
            if upsample_factor_x > 1:
                new_num_samples = sub_num_array.shape[0] * upsample_factor_x
                sub_num_array = resample(sub_num_array, new_num_samples)
                vid_num_array = resample(vid_num_array, new_num_samples)
            
            self.sub_num_data.append(sub_num_array)
            self.vid_num_data.append(vid_num_array)
     

        for file in y_files:
            y_df = pd.read_csv(file, usecols=y_columns)
            y_array = y_df.values[::downsampling_factor_y]
            
            if upsample_factor_y > 1:
                new_num_samples = y_array.shape[0] * upsample_factor_y
                y_array = resample(y_array.squeeze(), new_num_samples)
            
            self.y_data.append(y_array)
            
            
        for i in range(len(self.x_data)):
            # make sure they are the same length between x_file and y_files
            min_len = min(len(self.x_data[i]), len(self.y_data[i]))
            self.x_data[i] = self.x_data[i][:min_len]
            self.y_data[i] = self.y_data[i][:min_len]
            self.sub_num_data[i] = self.sub_num_data[i][:min_len]
            self.vid_num_data[i] = self.vid_num_data[i][:min_len]
            
#             if upsample_factor_x > 1:
#                 print(self.x_data[i].shape)
#                 print(self.y_data[i].shape)
#                 print(self.sub_num_data[i].shape)
#                 print(self.vid_num_data[i].shape)
            
            # Combine x and y data, drop rows with NaN values
            combined_df = pd.concat([pd.DataFrame(self.x_data[i], columns=x_columns),
                         pd.DataFrame(self.y_data[i], columns=y_columns),
                         pd.DataFrame(self.sub_num_data[i], columns=['sub_num_dataset']),
                         pd.DataFrame(self.vid_num_data[i], columns=['vid_num_dataset'])], axis=1)

            # Drop rows with NaN values
            combined_df = combined_df.dropna()

            # Extract cleaned x, y, sub_num, and vid_num data
            self.x_data[i] = combined_df[x_columns].values
            self.y_data[i] = combined_df[y_columns].values
            self.sub_num_data[i] = combined_df['sub_num_dataset'].values
            self.vid_num_data[i] = combined_df['vid_num_dataset'].values
  
    def upsample(self, x_array, upsample_factor_x):
        if x_array.shape[0] > 1:
            new_num_samples = x_array.shape[0] * upsample_factor_x
            col_mean = np.nanmean(x_array, axis=0)
            inds = np.where(np.isnan(x_array))
            x_array[inds] = np.take(col_mean, inds[1])
            x_array = resample(x_array.squeeze(), new_num_samples)
        elif x_array.shape[0] == 1 or x_array.ndim == 1:
            new_num_samples = x_array.shape[0] * upsample_factor_x
            col_mean = np.nanmean(x_array, axis=0)
            inds = np.where(np.isnan(x_array))
            x_array[inds] = np.take(col_mean, inds[1])
            x_array = resample(x_array, new_num_samples)
        return x_array
            
    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, index):
        x = self.x_data[index]
        y = self.y_data[index]
        sub_num = self.sub_num_data[index]
        vid_num = self.vid_num_data[index]

        return x, y, sub_num, vid_num


from sklearn.metrics import mean_squared_error, r2_score
import joblib
from math import sqrt
from scipy.signal import resample


def upsample_predictions(predictions, factor):
    """
    Upsample the predictions by a given factor using resample function.
    
    :param predictions: np.array of predictions
    :param factor: int, factor to upsample the predictions
    :return: np.array of upsampled predictions
    """
    new_num_samples = predictions.shape[0] * factor
    upsampled_predictions = resample(predictions.squeeze(), new_num_samples)
    
    return upsampled_predictions.reshape(-1, 1)

def TestModel(model, fold_data, x_columns, y_columns, save_file_name, 
                       
                      verbose = True):
    '''
    which_features: x column names
    which_y: y column name
    save_file_name: descriptive name for the specific version of this model.
    e.g., LinearReg_arousal_
    
    '''
    test_res = []
    all_pred_y = []
    all_sub_num = []
    all_vid_num = []
    for f, fold in enumerate(fold_data):
        test_x_files, test_y_files = fold[1]

        
        test_dataset = EmognitionDataset(
            test_x_files, test_y_files, x_columns=x_columns, y_columns=y_columns,
         upsample_factor_x=20, upsample_factor_y=1
        )
    

#         train_x = np.vstack([train_dataset[i][0] for i in range(len(train_dataset))])
#         train_y = np.vstack([train_dataset[i][1] for i in range(len(train_dataset))])

        test_x = np.vstack([test_dataset[i][0] for i in range(len(test_dataset))])
        test_y = np.vstack([test_dataset[i][1] for i in range(len(test_dataset))])
        #print([test_dataset[i][2] for i in range(len(test_dataset))])
        test_sub = np.hstack([test_dataset[i][2] for i in range(len(test_dataset))])
        test_vid = np.hstack([test_dataset[i][3] for i in range(len(test_dataset))])
        
#         s = 0
#         for i in range(len(test_dataset)):
#             s += test_dataset[i][0].shape[0]
#             print(s)
#         print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)
        

       # Make predictions
        pred_y = model.predict(test_x).reshape(-1, 1)
        
        # Upsample predictions
#         upsampled_pred_y = upsample_predictions(pred_y, downsampling_factor_y)

        # Calculate metrics with upsampled predictions
        rmse = sqrt(mean_squared_error(test_y, pred_y))
        test_res.append(rmse)
        r2 = r2_score(test_y, pred_y)
        
        
        if verbose:
            print(f'---------------------- fold {f+1} ---------------------------')
            print(f'rmse = {rmse:.3f}')
            print(f'r2 score = {r2:.3f}')

        # save: 
        all_pred_y.append(pred_y)
        all_sub_num.append(test_sub)
        all_vid_num.append(test_vid)

  #  return all_pred_y
    all_pred_y_concat = np.vstack(all_pred_y)
    all_sub_num_concat = np.hstack(all_sub_num)
    all_vid_num_concat = np.hstack(all_vid_num)

    all_pred_y_df = pd.DataFrame({'sub': all_sub_num_concat, 'vid': all_vid_num_concat, 'pred_y': all_pred_y_concat[:,0]})
    all_pred_y_df.to_csv(save_file_name + '.csv', index=False)


    joblib.dump(model, save_file_name + '.joblib')
    return test_res, all_pred_y_df, model

def train_model_no_eval(model, 
                        train_x_files,
                        train_y_files,
                        x_columns, 
                        y_columns, 
                        save_file_name, 
                        downsampling_factor_y=20, 
                        downsampling_factor_x=1, 
                        verbose = True):
    '''
    which_features: x column names
    which_y: y column name
    save_file_name: descriptive name for the specific version of this model.
    e.g., LinearReg_arousal_
    
    '''
    train_x_files, train_y_files

    train_dataset = EmognitionDataset(
        train_x_files, 
        train_y_files,
        x_columns=x_columns,
        y_columns=y_columns,
        downsampling_factor_y=downsampling_factor_y,
        downsampling_factor_x=downsampling_factor_x
    )

    train_x = np.vstack([train_dataset[i][0] for i in range(len(train_dataset))])
    train_y = np.vstack([train_dataset[i][1] for i in range(len(train_dataset))])
    model.fit(train_x, train_y)
    joblib.dump(model, save_file_name + '.joblib')
    return model
        




In [ ]:
import joblib

In [ ]:
#scenario 2

#random forest (fizz_emg) --> arousal
#SVR_RBF (fizz_emg)--> valence

# Scenario 2 valence elastic_net fizz-emg_sub_vid
# Scenario2 arousal rf fizz_sub-emg_sub_vid

for fold_num in range(0,5):        
    arousal_model_name = f'/work/abslab/emognition_2023_challenge/results/results_30s/scenario_2/fold_{fold_num}/random_forest_default_arousal_fizz-emg_sub_vid.joblib'
    valence_model_name = f'/work/abslab/emognition_2023_challenge/results/results_30s/scenario_2/fold_{fold_num}/Elastic_Net_valence_fizz-emg_sub_vid.joblib'
    
    arousal_model = joblib.load(arousal_model_name)
    valence_model = joblib.load(valence_model_name)
    
    test_data_folder =  f'/work/abslab/emognition_2023_challenge/file_prep_30sec/scenario_2/fold_{fold_num}/test/physiology/'
    annotations_folder = f'/work/abslab/emognition_2023_challenge/data/scenario_2/fold_{fold_num}/test/annotations/'
    
    save_data_folder = f'/work/abslab/emognition_2023_challenge/results/results_30s/scenario_2/fold_{fold_num}/test/'
    
    for file in os.listdir(test_data_folder):
        if file.endswith("_resampled_1hz.csv"):
            print(file)
            
            annotations_file_name = file[:-len("_resampled_1hz.csv")] + ".csv"
            annotations_file_ = os.path.join(annotations_folder, annotations_file_name)
            annotations_data = pd.read_csv(annotations_file_, index_col=0)
            annotations_len = annotations_data.shape[0]
            
            file_ = os.path.join(test_data_folder, file)
            
            test_data_arousal = pd.read_csv(file_, index_col=0)
            test_data_arousal = test_data_arousal[fizz_columns_of_interest_no_emg_sub_vid]
            if test_data_arousal.shape[0] == 1:
                test_data_arousal = np.nan_to_num(test_data_arousal)
            new_num_samples = annotations_len
            col_mean = np.nanmean(test_data_arousal, axis=0)
            inds = np.where(np.isnan(test_data_arousal))
            test_data_arousal[inds] = np.take(col_mean, inds[1])
            test_data_arousal = resample(test_data_arousal, new_num_samples)
            
            test_data_valence = pd.read_csv(file_, index_col=0)
            test_data_valence = test_data_valence[fizz_columns_of_interest_no_emg_sub_vid]
            if test_data_valence.shape[0] == 1:
                test_data_valence = np.nan_to_num(test_data_valence)
            new_num_samples = annotations_len
            col_mean = np.nanmean(test_data_valence, axis=0)
            inds = np.where(np.isnan(test_data_valence))
            test_data_valence[inds] = np.take(col_mean, inds[1])
            test_data_valence = resample(test_data_valence, new_num_samples)
            
            arousal_pred_y = arousal_model.predict(test_data_arousal).reshape(-1, 1)
            valence_pred_y = valence_model.predict(test_data_valence).reshape(-1, 1)
            
            annotations_data['valence'] = valence_pred_y
            annotations_data['arousal'] = arousal_pred_y
            
            save_file_ = os.path.join(save_data_folder, annotations_file_name)
            annotations_data.to_csv(save_file_)
            

In [ ]:
#scenario 3
#random forest (fizz_sub_video)--> arousal
#SVR_RBF (fizz_emg)--> valence


for fold_num in range(0,4):        
    arousal_model_name = f'/work/abslab/emognition_2023_challenge/results/results_30s/scenario_3/fold_{fold_num}/SVR_RBF_arousal_fizz.joblib'
    #valence_model_name = f'/work/abslab/emognition_2023_challenge/results/results_30s/scenario_3/fold_{fold_num}/SVR_RBF_valence_fizz-emg.joblib'
    
    arousal_model = joblib.load(arousal_model_name)
    #valence_model = joblib.load(valence_model_name)
    
    test_data_folder =  f'/work/abslab/emognition_2023_challenge/file_prep_30sec/scenario_3/fold_{fold_num}/test/physiology/'
    annotations_folder = f'/work/abslab/emognition_2023_challenge/results/final_submission_alessia/results/scenario_3/fold_{fold_num}/test/annotations/'
    
    save_data_folder = f'/work/abslab/emognition_2023_challenge/results/final_submission_alessia/results/scenario_3/fold_{fold_num}/test/annotations/'
    
    for file in os.listdir(test_data_folder):
        if file.endswith("_resampled_1hz.csv"):
            print(file)
            
            annotations_file_name = file[:-len("_resampled_1hz.csv")] + ".csv"
            annotations_file_ = os.path.join(annotations_folder, annotations_file_name)
            annotations_data = pd.read_csv(annotations_file_, index_col=0)
            annotations_len = annotations_data.shape[0]
            
            file_ = os.path.join(test_data_folder, file)
            
            test_data_arousal = pd.read_csv(file_, index_col=0)
            test_data_arousal = test_data_arousal[fizz_columns_of_interest]
            if test_data_arousal.shape[0] == 1:
                test_data_arousal = np.nan_to_num(test_data_arousal)
            new_num_samples = annotations_len
            col_mean = np.nanmean(test_data_arousal, axis=0)
            inds = np.where(np.isnan(test_data_arousal))
            test_data_arousal[inds] = np.take(col_mean, inds[1])
            test_data_arousal = resample(test_data_arousal, new_num_samples)
            
#             test_data_valence = pd.read_csv(file_, index_col=0)
#             test_data_valence = test_data_valence[fizz_columns_of_interest_no_emg]
#             if test_data_valence.shape[0] == 1:
#                 test_data_valence = np.nan_to_num(test_data_valence)
#             new_num_samples = annotations_len
#             col_mean = np.nanmean(test_data_valence, axis=0)
#             inds = np.where(np.isnan(test_data_valence))
#             test_data_valence[inds] = np.take(col_mean, inds[1])
#             test_data_valence = resample(test_data_valence, new_num_samples)
            
            arousal_pred_y = arousal_model.predict(test_data_arousal).reshape(-1, 1)
#             valence_pred_y = valence_model.predict(test_data_valence).reshape(-1, 1)
            
#             annotations_data['valence'] = valence_pred_y
            annotations_data['arousal'] = arousal_pred_y
            
            save_file_ = os.path.join(save_data_folder, annotations_file_name)
            annotations_data.to_csv(save_file_)
            


In [ ]:
#scenario 4

#random forest (fizz_emg_sub_video)---> arousal
# SVR (fizz) --> valence



for fold_num in range(0,3):        
    arousal_model_name = f'/work/abslab/emognition_2023_challenge/results/results_30s/scenario_4/fold_{fold_num}/SVR_RBF_arousal_fizz.joblib'
#     valence_model_name = f'/work/abslab/emognition_2023_challenge/results/results_30s/scenario_4/fold_{fold_num}/SVR_RBF_valence_fizz.joblib'
    
    arousal_model = joblib.load(arousal_model_name)
#     valence_model = joblib.load(valence_model_name)
    
    test_data_folder =  f'/work/abslab/emognition_2023_challenge/file_prep_30sec/scenario_4/fold_{fold_num}/test/physiology/'
    annotations_folder = f'/work/abslab/emognition_2023_challenge/results/final_submission_alessia/results/scenario_4/fold_{fold_num}/test/annotations/'
    
    save_data_folder = f'/work/abslab/emognition_2023_challenge/results/final_submission_alessia/results/scenario_4/fold_{fold_num}/test/annotations/'
     
    for file in os.listdir(test_data_folder):
        if file.endswith("_resampled_1hz.csv"):
            print(file)
            
            annotations_file_name = file[:-len("_resampled_1hz.csv")] + ".csv"
            annotations_file_ = os.path.join(annotations_folder, annotations_file_name)
            annotations_data = pd.read_csv(annotations_file_, index_col=0)
            annotations_len = annotations_data.shape[0]
            
            file_ = os.path.join(test_data_folder, file)
            
            test_data_arousal = pd.read_csv(file_, index_col=0)
            test_data_arousal = test_data_arousal[fizz_columns_of_interest]
            if test_data_arousal.shape[0] == 1:
                test_data_arousal = np.nan_to_num(test_data_arousal)
            new_num_samples = annotations_len
            col_mean = np.nanmean(test_data_arousal, axis=0)
            inds = np.where(np.isnan(test_data_arousal))
            test_data_arousal[inds] = np.take(col_mean, inds[1])
            test_data_arousal = resample(test_data_arousal, new_num_samples)
            
#             test_data_valence = pd.read_csv(file_, index_col=0)
#             test_data_valence = test_data_valence[fizz_columns_of_interest]
#             if test_data_valence.shape[0] == 1:
#                 test_data_valence = np.nan_to_num(test_data_valence)
#             new_num_samples = annotations_len
#             col_mean = np.nanmean(test_data_valence, axis=0)
#             inds = np.where(np.isnan(test_data_valence))
#             test_data_valence[inds] = np.take(col_mean, inds[1])
#             test_data_valence = resample(test_data_valence, new_num_samples)
            
            arousal_pred_y = arousal_model.predict(test_data_arousal).reshape(-1, 1)
#             valence_pred_y = valence_model.predict(test_data_valence).reshape(-1, 1)
            
#             annotations_data['valence'] = valence_pred_y
            annotations_data['arousal'] = arousal_pred_y
            
            save_file_ = os.path.join(save_data_folder, annotations_file_name)
            annotations_data.to_csv(save_file_)
            
